In [32]:
# Install the tdmclient package:
!pip3 install tdmclient --upgrade
!pip3 install numpy --upgrade

import tdmclient.notebook
import numpy as np
import math
from tdmclient import ClientAsync, aw
import math
from time import monotonic

client = ClientAsync()

# Physical constants of the Thymio
WHEELS_DIST_MM = 95.0
SPEED_MM_S = 200.0/500.0 #Calibrate per robot



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: c:\Users\Manuela\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: c:\Users\Manuela\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [ ]:
# Test Moving Control
from tdmclient import ClientAsync, aw
import heapq
import math as m
#import Global_Nav.global_nav as gb
#from Global_Nav.global_nav import debug_generate_maze, debug_maze_to_bitmap, a_star 
import importlib, Control_Thymio
import Control_Thymio
importlib.reload(Control_Thymio)
from Control_Thymio import follow_astar_path

# for debugging only 
import random 
from PIL import Image
# get a map somehow. For debugging I used a random generated map
#env_map = gb.debug_generate_maze(200, 200)
#gb.debug_maze_to_bitmap(env_map, 2)
#path = gb.a_star(env_map, m.pi)
#path = [(i-130, j-100) for (i,j) in path]
path = [(0,0), (1,0), (2,2), (3,2), (4,3)]

print(path)
#params
L = 113.5  
W = 81.3 
Y_res = 480                                  # number of rows (height)
Ratio = L / W                                # aspect ratio
SIZE = [Y_res, int(Y_res * Ratio)]  
SIZE_pixel = [Y_res, int(Y_res * Ratio)]

client = ClientAsync()
node = aw(client.wait_for_node())     # finds the wireless Thymio via TDM
try:
    aw(node.lock())                       # lock the node for R/W
except Exception:
    pass # ignore it it wasn't locked
    
        # Stop any running Aseba program so it doesn’t override your motor commands
aw(node.stop())
print("Connected:", node)
#aw(node.wait_for_variables({"motors.left.speed","motors.right.speed"}))

try:
    x, y, th = aw(follow_astar_path(
        node=node, client=client, path_cells=path,
        x0=0, y0=0, th0=0,
        cell_size_mm=20.0,
        linear_speed=150, angular_speed=150,
        dt=0.01, pos_tol=10.0, ang_tol=0.03
    ))
    print(f"Final pose (odometry): x={x:.1f} mm, y={y:.1f} mm, th={m.degrees(th):.1f} deg")
    pass
finally:
    #Makes sure the robot is stopped and the node available
    aw(node.set_variables({
        "motor.left.target": [0],
        "motor.right.target": [0]}))
    #aw(node.stop())
    aw(node.unlock())

[(0, 0), (1, 0), (2, 2), (3, 2), (4, 3)]
Connected: Node 30f587b4-fb96-452d-aa4f-57791877b058
Waypoints (mm): [(10.0, 10.0), (10.0, 30.0), (50.0, 50.0), (50.0, 70.0), (70.0, 90.0)]
Final pose (odometry): x=0.0 mm, y=0.0 mm, th=0.0 deg


KeyboardInterrupt: 

Exception in thread Thread-17:
Traceback (most recent call last):
  File "c:\Users\Manuela\anaconda3\envs\mobrob\Lib\threading.py", line 1075, in _bootstrap_inner
Exception in thread Thread-18:
Traceback (most recent call last):
  File "c:\Users\Manuela\anaconda3\envs\mobrob\Lib\threading.py", line 1075, in _bootstrap_inner
Exception in thread Thread-19:
Traceback (most recent call last):
  File "c:\Users\Manuela\anaconda3\envs\mobrob\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\Users\Manuela\anaconda3\envs\mobrob\Lib\site-packages\tdmclient\tcp.py", line 73, in run
    packet = self.read_packet()
             ^^^^^^^^^^^^^^^^^^
  File "c:\Users\Manuela\anaconda3\envs\mobrob\Lib\site-packages\tdmclient\tcp.py", line 66, in read_packet
    self.run()
  File "c:\Users\Manuela\anaconda3\envs\mobrob\Lib\site-packages\tdmclient\tcp.py", line 73, in run
    self.run()
  File "c:\Users\Manuela\anaconda3\envs\mobrob\Lib\site-packages\tdmclient\tcp.py", line 73, i

In [9]:
# Simple motion test: drive forward 1 s, then stop
from tdmclient import ClientAsync, aw

client = ClientAsync()
node = aw(client.wait_for_node())     # connect via Thymio Suite (wireless dongle)
aw(node.lock())                       # take control
aw(node.stop())                       # stop any running program on the robot
print("Connected:", node)

try:
    # (Optional) ensure speed variables are available for later use
    #aw(node.wait_for_variables({"motors.left.speed", "motors.right.speed"}))

    # Command motors: forward 150 for 1 second, then stop
    aw(node.set_variables({"motor.left.target":[150], "motor.right.target":[150]}))
    aw(client.sleep(1.0))
    aw(node.set_variables({"motor.left.target":[0], "motor.right.target":[0]}))

finally:
    # Always release the robot
    aw(node.set_variables({"motor.left.target":[0], "motor.right.target":[0]}))
    aw(node.unlock())

Connected: Node 30f587b4-fb96-452d-aa4f-57791877b058


In [36]:
# Get a node and lock it (might not be needed for the final implementation)
client = ClientAsync()
node = await client.wait_for_node()
await node.unlock()
#await tdmclient.notebook.stop()
#await node.lock()

await node.set_variables({"motor.left.target":[20], "motor.right.target":[20]}) 
await client.sleep(1.0) 
print("Measured speeds:", node.v.motors.left.speed, node.v.motors.right.speed) 
await node.set_variables({"motor.left.target":[0], "motor.right.target":[0]})

await node.unlock()

Measured speeds: <tdmclient.clientasynccachenode.VarPrefix object at 0x00000230D3896720> <tdmclient.clientasynccachenode.VarPrefix object at 0x00000230D3896390>


{'error_code': 2}

Exception in thread Exception in thread Thread-70:
Traceback (most recent call last):
  File "c:\Users\Manuela\anaconda3\envs\mobrob\Lib\threading.py", line 1075, in _bootstrap_inner
Exception in thread Thread-71:
Traceback (most recent call last):
  File "c:\Users\Manuela\anaconda3\envs\mobrob\Lib\threading.py", line 1075, in _bootstrap_inner
Exception in thread Thread-68:
Traceback (most recent call last):
  File "c:\Users\Manuela\anaconda3\envs\mobrob\Lib\threading.py", line 1075, in _bootstrap_inner
Exception in thread Thread-69:
Traceback (most recent call last):
  File "c:\Users\Manuela\anaconda3\envs\mobrob\Lib\threading.py", line 1075, in _bootstrap_inner
Exception in thread Thread-61:
Traceback (most recent call last):
  File "c:\Users\Manuela\anaconda3\envs\mobrob\Lib\threading.py", line 1075, in _bootstrap_inner
Exception in thread Thread-59:
Traceback (most recent call last):
  File "c:\Users\Manuela\anaconda3\envs\mobrob\Lib\threading.py", line 1075, in _bootstrap_inner
Ex